# Preprocessing Crime Data and ACS Data for 2005-2009
This notebook is designed to integrate preprocessed crime data with ACS (American Community Survey) data from 2005-2009. The process is different from integrating 2010 onwards crime/ACS data, as the ACS started in 2005 and API access with geographical identificators is not available for data before 2009. Refer to documentation for necessary data imports. 
The ACS data can be downloaded from from https://usa.ipums.org/usa-action/variables/group after creating a login. The readme contains the necessary tables
## Steps Overview:
1. Load ACS Data from 2005 to 2009 
2. Retrieve census tract ID from state, county and tract codes
3. Load shape files of pre-2010 census tracts from Chicago and map them to police districts using areal weighting
4. Aggregate the ACS 2005-2009 data to police district level
5. Load 2001 to 2024 crime data from CPD 
6. Filter Crime Data by Date Range 2005 to 2009
7. Check Police District Unique Categories
8. Derive 2005-2009 crime rates using ACS total population and crime dataset
9. Save files for further preprocessing (will be used in subsequent notebooks)

In [70]:
import pandas as pd
import geopandas as gpd
import numpy as np
from census import Census
from us import states
import pkg_resources 

In [72]:
print("pandas version:", pd.__version__)
print("geopandas version:", gpd.__version__)
print("numpy version:", np.__version__)

# Use pkg_resources to get the version for Census and states
print("census version:", pkg_resources.get_distribution("census").version)
print("states version:", pkg_resources.get_distribution("us").version)


pandas version: 2.0.3
geopandas version: 1.0.1
numpy version: 1.24.3
census version: 0.8.22
states version: 3.2.0


In [74]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

## Step 1: Load ACS Data from 2005 to 2009 

In [89]:
# Load ACS estimates from 2005-2009 (_E file)- this needs to be downloaded from https://usa.ipums.org/usa-action/variables/group after creating a login
acs2005_2009 = pd.read_csv('nhgis0006_ds195_20095_tract_E.csv', encoding='ISO-8859-1')


In [91]:
# Filter for Chicago (Cook County, Illinois)
acs2005_2009_chicago = acs2005_2009[(acs2005_2009['STATEA'] == 17) & (acs2005_2009['COUNTYA'] == 31)]

In [93]:
# Display the first few rows to confirm filtering
print(acs2005_2009_chicago.head(2))


              GISJOIN       YEAR STUSAB  REGIONA  DIVISIONA     STATE  STATEA  \
17770  G1700310000000  2005-2009     IL      NaN        NaN  Illinois      17   
17771  G1700310010100  2005-2009     IL      NaN        NaN  Illinois      17   

            COUNTY  COUNTYA  COUSUBA  ...  RLIE020  RLIE021  RNBE001  RNBE002  \
17770  Cook County       31      NaN  ...        0        0        0        0   
17771  Cook County       31      NaN  ...        0        0     4835      867   

       RNBE003  RNBE004  RNBE005  RNBE006  RNBE007  RNBE008  
17770        0        0        0        0        0        0  
17771      852      112      253      438       22     2291  

[2 rows x 68 columns]


In [95]:
# Check for missing values
print(acs2005_2009_chicago.isnull().sum())

# Option 1: Fill missing values with 0 (for numerical columns)
acs2005_2009_chicago.fillna(0, inplace=True)


GISJOIN         0
YEAR            0
STUSAB          0
REGIONA      1344
DIVISIONA    1344
             ... 
RNBE004         0
RNBE005         0
RNBE006         0
RNBE007         0
RNBE008         0
Length: 68, dtype: int64


## Step 2: Retrieve census tract ID from state, county and tract codes

In [98]:
# Now construct a unique identifier for census tracts by combining STATEA, COUNTYA, and TRACTA
acs2005_2009_chicago['census_tract_id'] = acs2005_2009_chicago['STATEA'].astype(str).str.zfill(2) + \
                               acs2005_2009_chicago['COUNTYA'].astype(str).str.zfill(3) + \
                               acs2005_2009_chicago['TRACTA'].astype(str).str.zfill(6)

## Step 3: Load shape files of pre-2010 census tracts from Chicago and map them to police districts using areal weighting
https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Census-Tracts-2000/pt6c-hxpp - tracts

https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Beats-effective-12-19-2012-/dq28-4w9c - districts



In [101]:
# Load the census tract and police districts shapefiles (you need the correct file path/ or downloaded file names)

tracts = gpd.read_file('Census_Tracts.shp')
districts = gpd.read_file('geo_export_04c70f5d-35cd-4d51-82f4-222f5c605f9a.shp')

In [103]:
# Display the columns in the districts GeoDataFrame
print(districts.columns)

# View the first few rows to get an idea of the data
print(districts.head())

Index(['dist_label', 'dist_num', 'geometry'], dtype='object')
  dist_label dist_num                                           geometry
0       17TH       17  POLYGON ((-87.71067 41.99737, -87.71067 41.997...
1       20TH       20  POLYGON ((-87.66029 41.99092, -87.66029 41.990...
2       31ST       31  POLYGON ((-87.82818 41.98384, -87.82816 41.983...
3       31ST       31  POLYGON ((-87.83365 41.97535, -87.83366 41.974...
4       19TH       19  POLYGON ((-87.64492 41.96973, -87.64431 41.969...


In [105]:
# Remove District 31 since it's not an official district despite appearing in the files
districts = districts[districts['dist_label'] != '31ST']
# Ensure District 31 is excluded
print(districts[districts['dist_label'] == '31ST'])  # This should return an empty DataFrame


Empty GeoDataFrame
Columns: [dist_label, dist_num, geometry]
Index: []


In [107]:
# Reproject census tracts and police districts/beats to a projected CRS for accurate area calculations
tracts = tracts.to_crs(epsg=26971)
districts = districts.to_crs(epsg=26971)

In [109]:
# Perform spatial intersection to get tracts intersecting districts
tracts_to_districts = gpd.overlay(tracts, districts, how='intersection')

# Calculate intersection area
tracts_to_districts['intersection_area'] = tracts_to_districts.geometry.area

# Calculate total area for each census tract
tracts_to_districts['total_tract_area'] = tracts_to_districts.groupby('CENSUS_T_1')['intersection_area'].transform('sum')

# Calculate the proportion of each tract within the district
tracts_to_districts['proportion'] = tracts_to_districts['intersection_area'] / tracts_to_districts['total_tract_area']

# Select relevant columns for the merge (use 'CENSUS_T_1' for the 2000 census tract identifier)
tracts_to_districts = tracts_to_districts[['CENSUS_T_1', 'dist_num', 'proportion']]

# Filter out any rows where dist_num is 31 (in case any remain in the result)
tracts_to_districts = tracts_to_districts[tracts_to_districts['dist_num'] != 31]
# Display the first few rows to verify
print(tracts_to_districts.head())

    CENSUS_T_1 dist_num    proportion
0  17031720500       22  1.000000e+00
1  17031730200       22  1.000000e+00
2  17031730300       22  1.000000e+00
3  17031740100       22  1.000000e+00
4  17031711200       22  2.530625e-07


## Step 4: Aggregate the ACS 2005-2009 data to police district level

In [111]:
# Merge the tract-to-district mapping with socio-economic data
# Merge on 'CENSUS_T_1' (from shapefile) and 'census_tract_id' (constructed in the acs2005_2009_chicago)
merged_data2005 = pd.merge(tracts_to_districts, acs2005_2009_chicago, left_on='CENSUS_T_1', right_on='census_tract_id')
# Columns to keep for calculating demographic percentages and poverty percentage
columns_to_keep = [
    'RK9E001',  # Total Population (from Table 1)
    'RLIE001',  # Total Population (for Hispanic or Latino Origin by Race)
    'RLIE003',  # Not Hispanic or Latino: White alone
    'RLIE004',  # Not Hispanic or Latino: Black or African American alone
    'RLIE006',  # Not Hispanic or Latino: Asian alone
    'RLIE012',  # Hispanic or Latino
    'RNBE001',  # Total population for whom poverty status is determined (from Table 3)
    'RNBE002',  # Population under 0.50 (extreme poverty)
    'RNBE003',  # Population between 0.50 to 0.99 (below poverty threshold)
    'dist_num',  # Police district number from the shapefile
    'proportion'  # Proportion of tract in the district
]

# Filter the merged dataset to keep only relevant columns
filtered_df = merged_data2005[columns_to_keep]

# Create new calculated columns for demographic percentages
filtered_df['Percent_Hispanic'] = (filtered_df['RLIE012'] / filtered_df['RK9E001']) * 100
filtered_df['Percent_NonHispanic_White'] = (filtered_df['RLIE003'] / filtered_df['RK9E001']) * 100
filtered_df['Percent_NonHispanic_Black'] = (filtered_df['RLIE004'] / filtered_df['RK9E001']) * 100
filtered_df['Percent_NonHispanic_Asian'] = (filtered_df['RLIE006'] / filtered_df['RK9E001']) * 100

# Calculate the poverty percentage:
# Poverty Percent = (Population under poverty threshold / Total Population) * 100
filtered_df['Poverty_Percent'] = ((filtered_df['RNBE002'] + filtered_df['RNBE003']) / filtered_df['RNBE001']) * 100

# Drop intermediate columns
filtered_df = filtered_df.drop(columns=[
    'RLIE003', 'RLIE004', 'RLIE006', 'RLIE012',  # Demographic-related columns
    'RNBE002', 'RNBE003'  # Poverty-related columns
])

# Rename remaining columns for clarity
filtered_df = filtered_df.rename(columns={
    'RK9E001': 'Total_Population',
    'RNBE001': 'Total_Population_Poverty_Status'
})

# Replace NaN values with 0
filtered_df.fillna(0, inplace=True)

# Final dataset will have Total Population, Poverty_Percent, and the calculated percentages for ethnic composition
print(filtered_df.head(10)) 



   Total_Population  RLIE001  Total_Population_Poverty_Status dist_num  \
0              2283     2283                             2283       22   
1              9411     9411                             9411       22   
2              1404     1404                             1404       22   
3              3198     3198                             3192       22   
4              5761     5761                             5761       22   
5              5761     5761                             5761        6   
6              2143     2143                             2129       22   
7              2143     2143                             2129        6   
8              4789     4789                             4778       22   
9              2093     2093                             2093       22   

     proportion  Percent_Hispanic  Percent_NonHispanic_White  \
0  1.000000e+00          4.029785                  94.918966   
1  1.000000e+00          0.563171                   0.690

In [114]:
# Define aggregation functions to get district-level statistics
agg_funcs = {
    'Total_Population': 'sum',              # Sum total population
    'Poverty_Percent': 'mean',              # Average poverty percentage
    'Percent_NonHispanic_White': 'mean',    # Average Non-Hispanic White percentage
    'Percent_NonHispanic_Black': 'mean',    # Average Non-Hispanic Black percentage
    'Percent_NonHispanic_Asian': 'mean',    # Average Non-Hispanic Asian percentage
    'Percent_Hispanic': 'mean',             # Average Hispanic percentage
}

# Group by dist_num and aggregate using the functions above
district_aggregated2005 = filtered_df.groupby('dist_num').agg(agg_funcs).reset_index()

# Display the aggregated district-level data
print(district_aggregated2005.head())

  dist_num  Total_Population  Poverty_Percent  Percent_NonHispanic_White  \
0        1             95717        19.351846                  37.005612   
1       10            142736        32.853045                   6.826244   
2       11            132036        36.099363                   4.769434   
3       12            200158        26.834720                  31.523792   
4       14            222914        20.430424                  41.621459   

   Percent_NonHispanic_Black  Percent_NonHispanic_Asian  Percent_Hispanic  
0                  35.439539                  13.647250          5.051153  
1                  44.298015                   0.750258         41.147614  
2                  75.509389                   1.190795         13.185960  
3                  34.011380                   8.668835         22.125298  
4                  10.730775                   2.949824         42.373288  


## Step 5: Load 2001 to 2024 crime data from CPD 

In [117]:
#load full crime data #replace with correct data file name
crime_data = pd.read_parquet('crimeunits2409.parquet', engine="pyarrow")

In [118]:
# Convert 'Date' column to datetime format
crime_data['Date'] = pd.to_datetime(crime_data['Date'], errors='coerce')
# Extract year and month
crime_data['year'] = crime_data['Date'].dt.year
crime_data['month'] = crime_data['Date'].dt.month

# Check for any rows where the 'Date' conversion failed
print(crime_data[crime_data['Date'].isna()].head())

Empty DataFrame
Columns: [Date, ID, Beat, District, Ward, Arrest, Latitude, Longitude, Primary Type, Crime_Category, year, month]
Index: []


## Step 6: Filter Crime Data by Date Range 2005 to 2009

In [120]:
# Filter the crime data for years 2004-2009
crime_filtered2005 = crime_data[(crime_data['year'] >= 2005) & (crime_data['year'] <= 2009)]


## Step 7: Check Police District Unique Categories

In [122]:
#The CPD data has been updated to reflect the latest district boundaries along the whole period
print("Unique districts in crime_data:", crime_filtered2005['District'].unique())

Unique districts in crime_data: ['024' '006' '002' '004' '005' '011' '010' '008' '022' '019' '025' '012'
 '001' '020' '003' '015' '016' '018' '017' '007' '009' '014']


In [124]:
# Remove leading zeros from the 'District' column in the crime data
crime_filtered2005['District'] = crime_filtered2005['District'].str.lstrip('0')

# Verify that the formats are now aligned
print(district_aggregated2005['dist_num'].unique())
print(crime_filtered2005['District'].unique())

['1' '10' '11' '12' '14' '15' '16' '17' '18' '19' '2' '20' '22' '24' '25'
 '3' '4' '5' '6' '7' '8' '9']
['24' '6' '2' '4' '5' '11' '10' '8' '22' '19' '25' '12' '1' '20' '3' '15'
 '16' '18' '17' '7' '9' '14']


## Step 8: Derive 2005-2009 crime rates using ACS total population and crime dataset

In [127]:
#Merge the ACS data with the crime data to get total population per district
crime_aggregated_monthly2005 = crime_filtered2005.groupby(['District', 'year', 'month', 'Crime_Category']).size().unstack(fill_value=0).reset_index()

crime_aggregated_monthly2005 = crime_aggregated_monthly2005.merge(
    district_aggregated2005[['dist_num', 'Total_Population']], 
    how='left',
    left_on=['District'],
    right_on=['dist_num']
)

# Drop redundant 'dist_num'
crime_aggregated_monthly2005.drop(columns=['dist_num'], inplace=True)


In [129]:
# Calculate monthly crime rates per 1,000 people for different crime categories
crime_types = ['Violent Crime', 'Property Crime', 'Drug-Related Crime', 'Administrative or Non-Criminal', 'Public Order Crime', 'Other']

for crime_type in crime_types:
    crime_aggregated_monthly2005[f'{crime_type}_rate'] = (crime_aggregated_monthly2005[crime_type] / crime_aggregated_monthly2005['Total_Population']) * 1000

#Inspect the df with monthly crime rates
print(crime_aggregated_monthly2005.head())

  District  year  month  Administrative or Non-Criminal  Drug-Related Crime  \
0        1  2005      1                              84                 195   
1        1  2005      2                              73                 166   
2        1  2005      3                              82                 142   
3        1  2005      4                              75                 105   
4        1  2005      5                              71                 143   

   Other  Property Crime  Public Order Crime  Violent Crime  Total_Population  \
0      0            1080                  15            233             95717   
1      0             799                  11            193             95717   
2      0             920                  22            246             95717   
3      0             944                  18            268             95717   
4      0            1044                  22            271             95717   

   Violent Crime_rate  Property Crime_

In [131]:
#Inspect the df
print(crime_aggregated_monthly2005.tail())

     District  year  month  Administrative or Non-Criminal  \
1315        9  2009      8                             112   
1316        9  2009      9                              96   
1317        9  2009     10                             100   
1318        9  2009     11                              97   
1319        9  2009     12                              87   

      Drug-Related Crime  Other  Property Crime  Public Order Crime  \
1315                 232      0             894                  77   
1316                 228      0             784                  83   
1317                 226      0             809                  96   
1318                 202      0             809                  62   
1319                 179      0             724                  45   

      Violent Crime  Total_Population  Violent Crime_rate  \
1315            529            247011            2.141605   
1316            520            247011            2.105169   
1317            4

In [133]:
#Calculate the total crime rate by summing all individual crime category rates
crime_aggregated_monthly2005['total_crime_rate'] = (
    crime_aggregated_monthly2005[['Violent Crime_rate', 'Property Crime_rate', 'Drug-Related Crime_rate',
                              'Administrative or Non-Criminal_rate', 'Public Order Crime_rate', 'Other_rate']].sum(axis=1)
)

In [134]:
crime_aggregated_monthly2005.tail()

,District,year,month,Administrative or Non-Criminal,Drug-Related Crime,Other,Property Crime,Public Order Crime,Violent Crime,Total_Population,Violent Crime_rate,Property Crime_rate,Drug-Related Crime_rate,Administrative or Non-Criminal_rate,Public Order Crime_rate,Other_rate,total_crime_rate
1315,9,2009,8,112,232,0,894,77,529,247011,2.141605,3.619272,0.939229,0.453421,0.311727,0.0,7.465255
1316,9,2009,9,96,228,0,784,83,520,247011,2.105169,3.173948,0.923036,0.388647,0.336017,0.0,6.926817
1317,9,2009,10,100,226,0,809,96,425,247011,1.720571,3.275158,0.914939,0.404840,0.388647,0.0,6.704155
1318,9,2009,11,97,202,0,809,62,422,247011,1.708426,3.275158,0.817777,0.392695,0.251001,0.0,6.445057
1319,9,2009,12,87,179,0,724,45,370,247011,1.497909,2.931044,0.724664,0.352211,0.182178,0.0,5.688006


## Step 9: Save files for further preprocessing (will be used in subsequent notebooks)

In [141]:
#Save to a Parquet file if needed
crime_aggregated_monthly2005.to_parquet('crime_2005-2009.parquet', index=False)

In [66]:
district_aggregated2005.to_parquet('acs_2005-2009.parquet', index=False)